In [1]:
from pyathena.tigress_ncr.load_sim_tigress_ncr import LoadSimTIGRESSNCRAll

# load classic models
classic_models = dict(CL_R2='/tigress/changgoo/TIGRESS-default/R2_2pc_L512_B2_norun',
                      CL_LGR2='/tigress/changgoo/TIGRESS-default/LGR2_2pc_L512_B10',
                      CL_R4='/tigress/changgoo/TIGRESS-default/R4_2pc_L512_B10',
                      CL_LGR4='/tigress/changgoo/TIGRESS-default/LGR4_2pc_L512_B10',
                      CL_R8='/tigress/changgoo/TIGRESS-default/R8_4pc_newacc',
                      CL_LGR8='/tigress/changgoo/TIGRESS-default/LGR8_4pc_FUVcorr',
                      CL_R16='/tigress/changgoo/TIGRESS-default/R16_8pc_metal',
                      F20B10="/tigress/changgoo/TIGRESS-ARM/R8_Beta10_F20_two")

sa = LoadSimTIGRESSNCRAll(classic_models)

In [3]:
for mdl in ["CL_R2","CL_R4","CL_R8","F20B10"]:
    s = sa.set_model(mdl)
    print(f"-------- {mdl} ---------")
    h = s.read_hst()
    zp = s.load_zprof_for_sfr()

    torb = 2*np.pi/s.par['problem']['Omega'] # orbit time in code units
    torb_Myr = 2*np.pi/s.par['problem']['Omega']*s.u.Myr # orbit time in Myr

    dz = s.domain["dx"][2]
    print("z_max",s.domain["re"][2])
    print("dx",dz)
    print("rho_dm",s.par["problem"]["rhodm"])
    print("Sigma_star",s.par["problem"]["SurfS"])
    print("Sigma_gas,init",s.par["problem"]["surf"])
    print("torb", torb_Myr)
    # time range adopted in Kim, C.-G. et al. (2020a)
    if mdl.startswith("CL"):
        tmin = 0.5*torb
        tmax = 1.5*torb
    else:
        tmin = 1*torb
        tmax = 3*torb
    for f in ["Sigma_gas","sfr10","nmid"]:
        print(f,h[f][tmin:tmax].mean())

    # time range in the physical units
    tmin *= s.u.Myr
    tmax *= s.u.Myr

    Pmid = zp["Ptot"].sel(z=slice(-dz,dz),time=slice(tmin,tmax))
    Amid = zp["A"].sel(z=slice(-dz,dz),time=slice(tmin,tmax))
    nmid = zp["d"].sel(z=slice(-dz,dz),time=slice(tmin,tmax)).mean(dim=["time","z"])
    Pmid = (Pmid/Amid).mean(dim=["time","z"])
    H = np.sqrt((zp["d"]*zp["z"]**2).sum(dim="z")/zp["d"].sum(dim="z")).sel(time=slice(tmin,tmax)).mean(dim="time")
    print("nmid",nmid.sel(phase="whole").data)
    print("Pmid",Pmid.sel(phase="whole").data)
    print("Hgas",H.sel(phase="whole").data)

-------- CL_R2 ---------
z_max 1792
dx 2.0
rho_dm 0.08
Sigma_star 450
Sigma_gas,init 150.0
torb 61.4364972073922
Sigma_gas 74.02351922787534
sfr10 0.7708589153684954
nmid 7.658962660179527
nmid 7.6572906308358615
Pmid 1777681.546981003
Hgas 329.3303600670691
-------- CL_R4 ---------
z_max 1792
dx 2.0
rho_dm 0.0239821
Sigma_star 208.028
Sigma_gas,init 50.0
torb 114.34045248829767
Sigma_gas 29.518406076588384
sfr10 0.0910557698063678
nmid 1.3846779936627573
nmid 1.38288229607045
Pmid 270756.00051718793
Hgas 338.7227622774105
-------- CL_R8 ---------
z_max 3584
dx 4.0
rho_dm 0.0064
Sigma_star 42.0
Sigma_gas,init 12.0
torb 219.41606145497212
Sigma_gas 10.71759085034867
sfr10 0.005093944762083659
nmid 0.8620180260552153
nmid 0.8618135589872514
Pmid 19249.209174437015
Hgas 354.755199926887
-------- F20B10 ---------
z_max 3141.6
dx 12.271875
rho_dm 0.0064
Sigma_star 42.0
Sigma_gas,init 13.0
torb 204.78832402464067
Sigma_gas 9.681756572919108
sfr10 0.004358845994408396
nmid 0.2480186794941582
